In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv('/content/drive/MyDrive/ScrappedNews/Stock_news.csv')
data.head()

,Unnamed: 0,newsTitle,timestamp,url_source,content,source,author,urlToImage
0,0,Work is getting really weird,2024-07-01T12:23:51Z,https://www.businessinsider.com/shadow-stand-i...,Lorenzo Matteucci for BI\r\n<ul><li>This post ...,Business Insider,Dan DeFrancesco,https://i.insider.com/6647bce220abc1efe8fb4f96...
1,1,"Roaring Kitty lawsuit, GDP forecasts dip: Mond...",2024-07-01T21:47:31Z,https://finance.yahoo.com/video/roaring-kitty-...,Yahoo Finance Reporter Josh Schafer joins Aski...,Yahoo Entertainment,Josh Lipton,https://s.yimg.com/ny/api/res/1.2/X35nbQ9PzBBH...
2,2,Here's how you can pay zero federal taxes on a...,2024-07-01T10:50:00Z,https://finance.yahoo.com/news/heres-pay-zero-...,Here's how you can pay zero federal taxes on a...,Yahoo Entertainment,Christy Bieber,https://media.zenfs.com/en/moneywise_327/d6d5b...
3,3,"Transcript: Eva Shang, CEO, Legalist",2024-07-02T12:00:05Z,https://ritholtz.com/2024/07/transcript-eva-sh...,"The transcript from this weeks, MiB: Eva Shang...",Ritholtz.com,Barry Ritholtz,https://ritholtz.com/wp-content/uploads/2023/0...
4,4,New twist in clash between US short seller Hin...,2024-07-02T15:59:13Z,https://www.aljazeera.com/economy/2024/7/2/new...,Hindenburg Research has denied allegations by ...,Al Jazeera English,Al Jazeera,https://www.aljazeera.com/wp-content/uploads/2...


In [15]:
text = data.loc[1, 'content']

In [16]:
print(text)

Yahoo Finance Reporter Josh Schafer joins Asking for a Trend to discuss his key takeaways from the trading day


In [5]:
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 1.4 MB/s eta 0:00:00


In [6]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()


In [7]:
# Define a function to get sentiment scores
def get_sentiment_scores(text):
    scores = analyzer.polarity_scores(text)
    return scores['compound']

In [8]:
print(get_sentiment_scores(data['content'].head(1)))

0    0.0
1    1.0
2    0.0
3    0.0
dtype: float64


In [ ]:
import re

# Define a function to clean the text
def clean(text):
# Removes all special characters and numericals leaving the alphabets
    text = re.sub('[^A-Za-z]+', ' ', str(text))
    return text

In [ ]:
import nltk

"""This punkt tokenizer divides a text into a list of sentences by using an unsupervised algorithm to build a model for abbreviation words,
collocations, and words that start sentences. """

nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet

In [ ]:
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

# POS tagger dictionary
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    #print(tags)
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
          newlist.append(tuple([word, pos_dict.get(tag[0])]))
          #print(tag[0])
          #print(pos_dict.get(tag[0]))
    return newlist

df['POS tagged'] = df['Cleaned Reviews'].apply(token_stop_pos)
df.head()

In [ ]:
# Obtaining the stem words – Lemmatization

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
     if not pos:
        lemma = word
        lemma_rew = lemma_rew + " " + lemma
     else:
        lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
        lemma_rew = lemma_rew + " " + lemma
    return lemma_rew

df['Lemma'] = df['POS tagged'].apply(lemmatize)
df.head()

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


# function to calculate vader sentiment
def vadersentimentanalysis(review):
    vs = analyzer.polarity_scores(review)
    return vs['compound']

df['Sentiment'] = df['Lemma'].apply(vadersentimentanalysis)

# function to analyse
def vader_analysis(compound):
    if compound >= 0.5:
        return 'Positive'
    elif compound < 0 :
        return 'Negative'
    else:
        return 'Neutral'
df['Analysis'] = df['Sentiment'].apply(vader_analysis)
df.head()